In [1]:
pip install selenium webdriver_manager schedule

Note: you may need to restart the kernel to use updated packages.


In [2]:
def pipeline_steam():
    # CELLA 2: Import e scraping della pagina Steam
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import re
    import requests
    import pandas as pd
    import time
    from datetime import datetime
    import os.path
    import numpy as np
    
    print("Avvio scraping pagina Steam...")
    
    # Setup WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    
    url = "https://store.steampowered.com/charts/mostplayed"
    
    try:
        # Apri la pagina
        driver.get(url)
        
        # Aspetta che il contenuto sia caricato
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/app/730/')]"))
        )
        
        # Salva l'HTML della pagina
        html_content = driver.page_source
        print("✓ Pagina caricata con successo")
        
    finally:
        # Chiudi il browser
        driver.quit()
    
    # Estrai gli ID dei giochi
    pattern = r"/app/(\d+)/"
    extracted_ids = re.findall(pattern, html_content)
    extracted_ids = list(set(extracted_ids))  # Rimuovi duplicati
    print(f"✓ Trovati {len(extracted_ids)} giochi unici")
    
    # CELLA 3: Raccolta dati giocatori e nomi giochi
    # Dizionari per memorizzare i dati
    game_names = {}
    today_data = {}
    
    # Definisci la data di oggi
    today = pd.Timestamp.today().strftime('%Y-%m-%d')
    
    # Definisci i file
    csv_file = "steam_players_data.csv"
    csv_finale = "steam_players_data_finale.csv"
    
    print(f"\nRaccolta dati per {today}...")
    print("=" * 60)
    
    # Raccogli i dati per ogni app_id
    for i, app_id in enumerate(extracted_ids, 1):
        try:
            print(f"[{i}/{len(extracted_ids)}] AppID {app_id}...", end=" ")
            
            # 1. Recupera il numero di giocatori
            api_url = f"http://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid={app_id}&format=json"
            response = requests.get(api_url, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                player_count = data.get("response", {}).get("player_count", np.nan)
                today_data[app_id] = player_count
            else:
                print(f"⚠ Errore player count: {response.status_code}")
                today_data[app_id] = np.nan
            
            # 2. Recupera il nome del gioco
            store_api_url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
            store_response = requests.get(store_api_url, timeout=10)
            
            if store_response.status_code == 200:
                store_data = store_response.json()
                if store_data.get(app_id, {}).get("success"):
                    game_name = store_data[app_id]["data"]["name"]
                    game_names[app_id] = game_name
                    print(f"✓ {game_name}")
                else:
                    game_names[app_id] = f"Unknown_{app_id}"
                    print(f"⚠ Nome non disponibile")
            else:
                game_names[app_id] = f"Unknown_{app_id}"
                print(f"⚠ Errore nome gioco")
            
            # Ritardo per non sovraccaricare le API
            time.sleep(1.2)
            
        except Exception as e:
            print(f"✗ Errore: {e}")
            today_data[app_id] = np.nan
            game_names[app_id] = f"Error_{app_id}"
    
    print("\n" + "=" * 60)
    print("✓ Raccolta dati completata!\n")
    
    # CELLA 4: Salvataggio e aggiornamento file CSV intermedio
    print("Aggiornamento file CSV...")
    
    # Crea DataFrame con i dati di oggi (AppID come indice, data come colonna)
    today_df = pd.DataFrame(today_data, index=[today]).transpose()
    today_df.columns = [today]
    today_df.index.name = 'AppID'
    
    # Gestione file CSV esistente
    if os.path.isfile(csv_file):
        # Carica il file esistente
        existing_df = pd.read_csv(csv_file, index_col='AppID')

        # 🆕 AGGIUNTA PER RIMOZIONE RIGHE DUPLICATE 🆕
        # Rimuove le righe con lo stesso AppID (indice), mantenendo la prima occorrenza
        existing_df = existing_df[~existing_df.index.duplicated(keep='first')] 
        # 🆕 FINE AGGIUNTA 🆕
        
        # Converti gli indici in stringhe per confronto corretto
        existing_df.index = existing_df.index.astype(str)
        today_df.index = today_df.index.astype(str)
        
        if today in existing_df.columns:
            print(f"⚠ I dati per {today} sono già presenti. Sovrascrivendo...")
            existing_df[today] = today_df[today]
            final_df = existing_df
        else:
            # Aggiungi nuovi AppID se non presenti
            all_app_ids = set(existing_df.index) | set(today_df.index)
            
            # Crea un nuovo DataFrame con tutti gli AppID
            final_df = pd.DataFrame(index=sorted(all_app_ids))
            final_df.index.name = 'AppID'
            
            # Copia tutte le colonne esistenti
            for col in existing_df.columns:
                final_df[col] = existing_df[col]
            
            # Aggiungi la colonna di oggi
            final_df[today] = today_df[today]
            
            print(f"✓ Colonna {today} aggiunta al file esistente")
        
        # Salva il DataFrame aggiornato
        final_df.to_csv(csv_file)
    else:
        # Crea nuovo file
        today_df.to_csv(csv_file)
        print(f"✓ Nuovo file creato: {csv_file}")
    
    print(f"✓ File {csv_file} aggiornato con successo\n")
    
    # CELLA 5: Creazione file finale con nomi dei giochi
    print("Creazione file finale con nomi giochi...")
    
    # Leggi il file CSV aggiornato
    df = pd.read_csv(csv_file, index_col='AppID')
    
    # Trasponi il DataFrame (date come righe, giochi come colonne)
    df_trasposto = df.T
    
    # Rinomina le colonne usando il formato "Nome Gioco (AppID)"
    new_columns = {}
    for app_id in df_trasposto.columns:
        app_id_str = str(app_id)
        if app_id_str in game_names:
            # Pulisci il nome del gioco da caratteri problematici
            clean_name = game_names[app_id_str].replace(',', '').replace('"', '')
            new_columns[app_id] = f"{clean_name} ({app_id_str})"
        else:
            new_columns[app_id] = f"Unknown ({app_id_str})"
    
    df_trasposto = df_trasposto.rename(columns=new_columns)


    df_trasposto = df_trasposto.sort_values(
        by=df_trasposto.index[-1],  # Usa l'indice dell'ultima riga (la data di oggi)
        axis=1,                     # Ordina le COLONNE (i giochi)
        ascending=False,            # ORDINE DECRESCENTE (dal più alto al più basso)
        na_position='last'          # Metti i NaN alla fine
    )
    
    # Salva il file finale
    df_trasposto.to_csv(csv_finale)
    
    print(f"✓ File finale salvato: {csv_finale}")
    print(f"\nDimensioni: {df_trasposto.shape[0]} date × {df_trasposto.shape[1]} giochi")
    
    # CELLA 6: Visualizzazione risultati
    print("\n" + "=" * 60)
    print("ANTEPRIMA DEI DATI")
    print("=" * 60)
    
    print("\nPrime 5 colonne del file finale:")
    print(df_trasposto.iloc[:, :5])
    
    print("\n\nStatistiche per oggi:")
    today_stats = df_trasposto.loc[today].describe()
    print(today_stats)
    
    print(f"\n✓ Processo completato con successo!")
    print(f"  - File intermedio: {csv_file}")
    print(f"  - File finale: {csv_finale}")

In [ ]:
import schedule
import time
import random as r

r1 = r.randint(10, 60)
r2 = r.randint(10, 60)

schedule.every().day.at("10:54").do(pipeline_steam)
print("script avviato")
while True:
    print("avviamento schedule...")
    time.sleep(r1)
    schedule.run_pending()
    time.sleep(r2)
    print("schedule avviato...")

script avviato
avviamento schedule...
